In [ ]:
%pip install psycopg2-binary

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Thorough Data Analysis

In [ ]:
apps = {}

columns_to_keep = [
    "record_id", "input", "output", "tags",
    "groundedness_measure_with_cot_reasons",
    "relevance_with_cot_reasons",
    "qs_relevance_with_cot_reasons",
    "agreement_measure",
    "latency", "total_tokens", "total_cost"]

for app in tru.get_apps():
    app_id = app["app_id"]
    dfRecords, feedbackColumns = tru.get_records_and_feedback([app_id])
    apps[app_id] = dfRecords[columns_to_keep]

### Testing Normality

The Shapiro-Wilk test for normality, when used in the context of comparing two methods on the same set of test cases (especially in a paired test scenario), implies that each data point in one dataset corresponds to a data point in the other dataset. In other words, the data points are paired.

This is particularly relevant when you want to:

1. **Perform a Paired Sample Test**: In a paired sample t-test or a Wilcoxon signed-rank test, the difference between each pair of observations is crucial. These tests are based on the differences within each pair (i.e., each data point in Method 1 is subtracted from the corresponding data point in Method 2). Therefore, it"s important that the two datasets are aligned such that each data point in one dataset has a direct, corresponding data point in the other dataset.

1. **Test for Normality in Paired Differences**: When using the Shapiro-Wilk test in this context, you"re typically testing the normality of these differences, not the individual datasets. Therefore, the datasets must be paired correctly before calculating these differences.

For example, if you have a list of test cases and you apply Method 1 and Method 2 to each test case, you should ensure that the results for each method are aligned such that the result of Method 1 for Test Case 1 is in the same position (same index) as the result of Method 2 for Test Case 1, and so on for all test cases.

In summary, for paired analyses, the order and pairing of data points between datasets are crucial. The results for each test case from each method need to be correctly aligned for the paired analysis to be valid.

In [ ]:
apps.keys()

In [ ]:
# re-arrange data, and sort all dataFrames by the `input` column

dataFrames = {}

for app_id in apps:
    parts = app_id.split("_")
    tool = parts[0]
    dataset = "_".join(parts[3:])

    if dataset not in dataFrames:
        dataFrames[dataset] = {}

    dataFrames[dataset][tool] = apps[app_id].sort_values(by="input")

In [ ]:
import scipy.stats as stats

columns_to_test = [
    "groundedness_measure_with_cot_reasons",
    "relevance_with_cot_reasons",
    "qs_relevance_with_cot_reasons",
    "agreement_measure",
    "latency"]

for dataset in dataFrames:
    print(dataset)
    lc_df = dataFrames[dataset]["lc"]
    llama_df = dataFrames[dataset]["llama"]
    if len(lc_df) != len(llama_df):
        print("\tDataFrames do not have equal row counts, skipping :(")
        continue
    for test in columns_to_test:
        print(f"\tTesting normality of {test}:")
        lc_values = lc_df[test].to_list()
        llama_values = llama_df[test].to_list()

        # Calculate the differences
        differences = [x - y for x, y in zip(lc_values, llama_values)]

        # Perform the Shapiro-Wilk Test
        statistic, p_value = stats.shapiro(differences)

        print(f"\t\tShapiro-Wilk Test statistic: {statistic}")
        print("\t\tP-value:", p_value)

        # Interpretation
        alpha = 0.05
        if p_value > alpha:
            print("\t\t\tData follows a normal distribution (fail to reject H0)")
        else:
            print("\t\t\tData does NOT follow a normal distribution (reject H0)")

Conclusion: Most datasets comparisons do NOT follow a normal distribution

### Testing Betterness

**Wilcoxon Signed-Rank Test**: This is a non-parametric alternative to the paired sample t-test and is used to compare two related samples or repeated measurements on a single sample to assess whether their population mean ranks differ. It"s appropriate for your scenario where you have paired data (the same cases tested with two different methods).

In [ ]:
from scipy import stats
import numpy as np

columns_to_test = [
    "groundedness_measure_with_cot_reasons",
    "relevance_with_cot_reasons",
    "qs_relevance_with_cot_reasons",
    "agreement_measure",
    "latency"]

column_translation = {
    "groundedness_measure_with_cot_reasons": "groundedness",
    "relevance_with_cot_reasons" : "answer_relevance",
    "qs_relevance_with_cot_reasons" : "context_relevance",
    "agreement_measure" : "answer_correctness",
    "latency" : "latency"
}

for dataset in dataFrames:
    print(dataset)
    # Assuming lc_df and llama_df are paired datasets
    lc_df = dataFrames[dataset]["lc"]
    llama_df = dataFrames[dataset]["llama"]
    if len(lc_df) != len(llama_df):
        print("\tDataFrames do not have equal row counts, skipping :(")
        continue
    for test in columns_to_test:
        translated = column_translation[test]
        print(f"\tTesting differences of {translated}:")
        lc_values = lc_df[test].to_list()
        llama_values = llama_df[test].to_list()

        # Perform the Wilcoxon Signed-Rank Test
        stat, p = stats.wilcoxon(lc_values, llama_values)
        print("\t\tStatistics=%.3f, p=%.3f" % (stat, p))

        # Calculate the median of the differences
        differences = np.array(llama_values) - np.array(lc_values)
        median_difference = np.median(differences)
        print("\t\tMedian of Differences:", median_difference)

        # Interpretation
        alpha = 0.05
        if p > alpha:
            print("\t\t\tSame distribution (fail to reject H0)")
        else:
            print("\t\t\tDifferent distribution (reject H0)")
            if median_difference > 0:
                print("\t\t\t\tLlama generally scores higher.")
            elif median_difference < 0:
                print("\t\t\t\tLangchain generally scores higher.")
            else:
                print("\t\t\t\tNo difference in the median scores.")

